In [1]:
import os
import cv2
import numpy as np
import rootutils
from pathlib import Path
from dotenv import load_dotenv

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.data.components.utils import list_files, IMAGE_EXTENSIONS
from src.data.components.preprocessing.preproc_strategy_tile import sliding_window_with_coordinates

load_dotenv()

True

#### Step 1. Extract and align regions

In [ ]:
def extract_and_align_regions(image: np.array, mask: np.array, output_path: Path, image_name: str) -> None:
    # Ensure the mask is binary
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    # Compute the bounding box of the mask
    x, y, w, h = cv2.boundingRect(mask)
    # Compute the center of the bounding box
    bbox_cx, bbox_cy = x + w // 2, y + h // 2  # (x, y) coordinates
    # Create an empty black image of the same size
    H, W = image.shape[:2]
    output = np.zeros_like(image)
    # Extract the masked region from the original image
    masked_region = cv2.bitwise_and(image, image, mask=mask)
    # Compute the new center position (center of output image)
    new_cx, new_cy = W // 2, H // 2  # Image center
    # Compute translation offsets
    dx, dy = int(new_cx - bbox_cx), int(new_cy - bbox_cy)
    # Create a translation matrix
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    # Move the masked region to the new position
    moved_masked_region = cv2.warpAffine(masked_region, M, (W, H))
    # Move the mask itself to match the new position
    moved_mask = cv2.warpAffine(mask, M, (W, H))
    # Combine only the valid (non-zero) parts into the output image
    output[moved_mask > 0] = moved_masked_region[moved_mask > 0]
    # save the output image
    cv2.imwrite(str(output_path / image_name), output)

In [2]:
data_path = Path(os.environ.get('lear_good_data_path'))

In [ ]:
image_path = data_path / 'front'
mask_path = data_path / 'front_mask'
output_path = data_path / 'front_aligned'
output_path.mkdir(exist_ok=True, parents=True)

# Gather images
image_paths = list_files(image_path, file_extensions=IMAGE_EXTENSIONS)

# Run the extraction and alignment process
for image_path in image_paths:
    image = cv2.imread(str(image_path), cv2.IMREAD_COLOR)
    mask = cv2.imread(str(mask_path / image_path.name), cv2.IMREAD_GRAYSCALE)
    extract_and_align_regions(image, mask, output_path, image_path.name)

#### Step 2. Extract tiles in consistent regions

In [7]:
def is_background(image: np.ndarray, background_perc: float) -> bool:
    """
    Determines if an image has a background percentage of black pixels
    greater than the specified threshold.

    Args:
        background_perc (float): The threshold percentage for black pixels.

    Returns:
        bool: True if the percentage of black pixels is greater than the threshold, False otherwise.
    """
    black_pixels = np.all(image == 0, axis=-1)
    black_pixel_count = np.sum(black_pixels)
    total_pixels = image.shape[0] * image.shape[1]
    black_pixel_percentage = black_pixel_count / total_pixels
    return black_pixel_percentage > background_perc

In [ ]:
tile_path = data_path / 'front_tiles_512'
tile_path.mkdir(exist_ok=True, parents=True)

image_path = data_path / 'front_aligned'
image_paths = list_files(image_path, file_extensions=IMAGE_EXTENSIONS)

background_th = 0.8
tile_size = (512, 512)
overlap = 20

for img_path in image_paths:
    image = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    for tile, coordinates in sliding_window_with_coordinates(image, tile_size=tile_size, overlap=overlap):
        # calcaulte center of the tile
        x_st, y_st, x_en, y_en = coordinates
        cx = (x_st + x_en) // 2
        cy = (y_st + y_en) // 2
        # if tile has background, skip
        if is_background(tile, background_th):
            continue
        # make directory with tile center name
        tile_dir = tile_path / f'{cx}_{cy}'
        tile_dir.mkdir(exist_ok=True, parents=True)
        # save the tile
        cv2.imwrite(str(tile_dir / f'{img_path.stem}_tile.png'), tile)